In [1]:
import mysql.connector
import pandas as pd


### Connect to mySQL
To make it work, open your MySQL workbench

In [2]:
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="your_password" 
  
)
# i put a placeholder on password

In [3]:
def execute_query(conn, query):
    cursor = conn.cursor()
    print("Executing")
    cursor.execute(query)

    cursor.close()

In [4]:
database_name = 'main'
execute_query(conn, f"CREATE SCHEMA {database_name}")


Executing


In [5]:
conn= mysql.connector.connect(
    host="localhost",
    user="root",
    password="your_password", 
    database = database_name
  
)
#i put a placeholder in password

### Create Query Functions

In [6]:
def get_mysql_type(pandas_type):
    # Mappa i tipi di dati Pandas ai tipi di dati MySQL
    type_mapping = {
        'int64': 'INT',
        'float64': 'FLOAT',
        'object': 'VARCHAR(255)',  # Modifica la lunghezza a seconda delle tue esigenze
        'datetime64[ns]': 'DATETIME'
        # Aggiungi altri tipi di dati se necessario
    }

    return type_mapping.get(str(pandas_type), 'VARCHAR(255)')  # Tipo di default

In [7]:
def get_create_table_query(df, primary_keys, table_name, foreign_keys = None):
   
    # Costruire la query per creare la tabella
    create_table_query = f"CREATE TABLE {table_name} ("

    # Iterare sulle colonne del DataFrame
    for col_name, col_type in zip(df.columns, df.dtypes):
        # Ottieni il tipo di dati MySQL corrispondente al tipo della colonna nel DataFrame
        mysql_type = get_mysql_type(col_type)
        
        #fix forzato
        if col_name == 'blurb':
            mysql_type = 'TEXT'

        # Aggiungi la colonna e il tipo alla query di creazione della tabella
        create_table_query += f"{col_name} {mysql_type}, "

    # Aggiungi le primary keys alla query
    if primary_keys:
        create_table_query += f"PRIMARY KEY ({', '.join(primary_keys)}), "
            # Aggiungi le foreign keys alla query se specificate
    
    if foreign_keys:
        for foreign_key in foreign_keys:
            create_table_query += f"FOREIGN KEY ({foreign_key['column']}) REFERENCES {foreign_key['referenced_table']}({foreign_key['referenced_column']}), "
            
    create_table_query = create_table_query.rstrip(', ') + ")"
    return create_table_query

In [8]:
def execute_query(conn, query):
    cursor = conn.cursor()
    print("Executing")
    cursor.execute(query)

    cursor.close()

### Champion Name

In [9]:
df_champion_name = pd.read_excel('datasets_cleaned/champion_name.xlsx')
df_champion_name

,champion_ID,champion_name
0,266,Aatrox
1,103,Ahri
2,84,Akali
3,166,Akshan
4,12,Alistar
...,...,...
161,221,Zeri
162,115,Ziggs
163,26,Zilean
164,142,Zoe


In [10]:
champion_name_create_query = get_create_table_query(
    df_champion_name, ['champion_ID'], 'champion'
)
champion_name_create_query

'CREATE TABLE champion (champion_ID INT, champion_name VARCHAR(255), PRIMARY KEY (champion_ID))'

In [11]:
execute_query(conn, champion_name_create_query)

Executing


### Champion Base Info

In [12]:
df_champion_base_info = pd.read_excel('datasets_cleaned/champion_base_info.xlsx')
df_champion_base_info

,champion_ID,title,blurb,partype,attack,defense,magic,difficulty
0,266,the Darkin Blade,Once honored defenders of Shurima against the ...,Blood Well,8,4,3,4
1,103,the Nine-Tailed Fox,Innately connected to the magic of the spirit ...,Mana,3,4,8,5
2,84,the Rogue Assassin,Abandoning the Kinkou Order and her title of t...,Energy,5,3,8,7
3,166,the Rogue Sentinel,"Raising an eyebrow in the face of danger, Aksh...",Mana,0,0,0,0
4,12,the Minotaur,Always a mighty warrior with a fearsome reputa...,Mana,6,9,5,7
...,...,...,...,...,...,...,...,...
161,221,The Spark of Zaun,"A headstrong, spirited young woman from Zaun's...",Mana,8,5,3,6
162,115,the Hexplosives Expert,"With a love of big bombs and short fuses, the ...",Mana,2,4,9,4
163,26,the Chronokeeper,"Once a powerful Icathian mage, Zilean became o...",Mana,2,5,8,6
164,142,the Aspect of Twilight,"As the embodiment of mischief, imagination, an...",Mana,1,7,8,5


In [13]:
foreign_keys = [
    {'column': 'champion_ID', 'referenced_table': 'champion', 'referenced_column': 'champion_ID'}
]

champion_base_info_create_query = get_create_table_query(df_champion_base_info,
                                                         ['champion_ID'],
                                                         'champion_base_info',
                                                         foreign_keys)

champion_base_info_create_query

'CREATE TABLE champion_base_info (champion_ID INT, title VARCHAR(255), blurb TEXT, partype VARCHAR(255), attack INT, defense INT, magic INT, difficulty INT, PRIMARY KEY (champion_ID), FOREIGN KEY (champion_ID) REFERENCES champion(champion_ID))'

In [14]:
execute_query(conn, champion_base_info_create_query)

Executing


### Champion Base Stats

In [15]:
df_champion_base_stats = pd.read_excel('datasets_cleaned/champion_base_stats.xlsx')
df_champion_base_stats

,champion_ID,hp,hpperlevel,mp,mpperlevel,movespeed,armor,armorperlevel,spellblock,spellblockperlevel,...,hpregen,hpregenperlevel,mpregen,mpregenperlevel,crit,critperlevel,attackdamage,attackdamageperlevel,attackspeedperlevel,attackspeed
0,266,650,114,0,0.0,345,38,4.45,32,2.05,...,3.00,1.00,0.00,0.00,0,0,60,5.00,2.500,0.651
1,103,590,96,418,25.0,330,21,4.70,30,1.30,...,2.50,0.60,8.00,0.80,0,0,53,3.00,2.200,0.668
2,84,570,119,200,0.0,345,23,4.70,37,2.05,...,9.00,0.90,50.00,0.00,0,0,62,3.30,3.200,0.625
3,166,630,107,350,40.0,330,26,4.70,30,1.30,...,3.75,0.65,8.20,0.70,0,0,52,3.00,4.000,0.638
4,12,685,120,350,40.0,330,47,4.70,32,2.05,...,8.50,0.85,8.50,0.80,0,0,62,3.75,2.125,0.625
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
161,221,630,110,250,45.0,330,24,4.20,30,1.30,...,3.25,0.70,6.00,0.80,0,0,53,2.00,2.000,0.658
162,115,606,106,480,23.5,325,18,4.50,30,1.30,...,6.50,0.60,8.00,0.80,0,0,55,3.10,2.000,0.656
163,26,574,96,452,50.0,335,24,5.00,30,1.30,...,5.50,0.50,11.35,0.80,0,0,52,3.00,2.130,0.658
164,142,630,106,425,25.0,340,21,4.70,30,1.30,...,7.50,0.60,8.00,0.65,0,0,58,3.30,2.500,0.658


In [16]:
foreign_keys = [
    {'column': 'champion_ID', 'referenced_table': 'champion', 'referenced_column': 'champion_ID'}
]

champion_base_stats_create_query = get_create_table_query(df_champion_base_stats,
                                                         ['champion_ID'],
                                                         'champion_base_stats',
                                                         foreign_keys)

champion_base_stats_create_query

'CREATE TABLE champion_base_stats (champion_ID INT, hp INT, hpperlevel INT, mp INT, mpperlevel FLOAT, movespeed INT, armor INT, armorperlevel FLOAT, spellblock INT, spellblockperlevel FLOAT, attackrange INT, hpregen FLOAT, hpregenperlevel FLOAT, mpregen FLOAT, mpregenperlevel FLOAT, crit INT, critperlevel INT, attackdamage INT, attackdamageperlevel FLOAT, attackspeedperlevel FLOAT, attackspeed FLOAT, PRIMARY KEY (champion_ID), FOREIGN KEY (champion_ID) REFERENCES champion(champion_ID))'

In [17]:
execute_query(conn, champion_base_stats_create_query)

Executing


### Tags and Champion Tags

In [18]:
df_tag = pd.read_excel('datasets_cleaned/tag.xlsx')
df_tag

,tag_ID,tag_name
0,0,Fighter
1,1,Tank
2,2,Mage
3,3,Assassin
4,4,Marksman
5,5,Support


In [19]:
tag_create_query = get_create_table_query(df_tag, ['tag_ID'], 'tag')
tag_create_query

'CREATE TABLE tag (tag_ID INT, tag_name VARCHAR(255), PRIMARY KEY (tag_ID))'

In [20]:
execute_query(conn, tag_create_query)

Executing


In [21]:
df_champion_tags = pd.read_excel('datasets_cleaned/champion_tags.xlsx')
df_champion_tags

,champion_ID,tag_ID
0,266,0
1,266,1
2,103,2
3,103,3
4,84,3
...,...,...
283,26,2
284,142,2
285,142,5
286,143,2


In [22]:
foreign_keys = [
    {'column': 'champion_ID', 'referenced_table': 'champion', 'referenced_column': 'champion_ID'},
    {'column': 'tag_ID', 'referenced_table': 'tag', 'referenced_column': 'tag_ID'}

]

In [23]:
champion_tags_create_query = get_create_table_query(
    df_champion_tags, ['champion_ID', 'tag_ID'], 'champion_tags', foreign_keys
)

champion_tags_create_query

'CREATE TABLE champion_tags (champion_ID INT, tag_ID INT, PRIMARY KEY (champion_ID, tag_ID), FOREIGN KEY (champion_ID) REFERENCES champion(champion_ID), FOREIGN KEY (tag_ID) REFERENCES tag(tag_ID))'

In [24]:
execute_query(conn, champion_tags_create_query)

Executing


### Lane and Ranking

In [25]:
df_lane = pd.read_excel('datasets_cleaned/lane.xlsx')
df_lane

,lane_ID,lane_name
0,1,top
1,2,jungle
2,3,mid
3,4,adc
4,5,support


In [26]:
lane_create_query = get_create_table_query(df_lane, ['lane_ID'], 'lane')
lane_create_query

'CREATE TABLE lane (lane_ID INT, lane_name VARCHAR(255), PRIMARY KEY (lane_ID))'

In [27]:
execute_query(conn, lane_create_query)

Executing


In [28]:
df_ranking = pd.read_excel('datasets_cleaned/ranking.xlsx')
df_ranking

,ranking_ID,ranking_name,ranking_value
0,0,all,0
1,1,challenger,1
2,2,master,2
3,3,diamond,3
4,4,emerald,4
5,5,platinum,5
6,6,gold,6
7,7,silver,7
8,8,bronze,8
9,9,iron,9


In [29]:
ranking_create_query = get_create_table_query(df_ranking, ['ranking_ID'], 'ranking')
ranking_create_query

'CREATE TABLE ranking (ranking_ID INT, ranking_name VARCHAR(255), ranking_value INT, PRIMARY KEY (ranking_ID))'

In [30]:
execute_query(conn, ranking_create_query)

Executing


### Game Stats

In [31]:
df_game_stats = pd.read_excel('datasets_cleaned/game_stats.xlsx')
df_game_stats

,position,win_rate,pick_rate,ban_rate,pbi,games_played,grade,lane_ID,ranking_ID,champion_ID
0,1,50.20,10.21,19.44,3,1737071,0.888667,1,0,24
1,2,51.50,5.32,15.35,9,905128,1.111000,1,0,420
2,3,52.42,5.85,6.50,15,995476,1.222000,1,0,75
3,4,51.00,4.89,7.75,5,832573,1.222000,1,0,114
4,5,52.14,8.33,6.26,19,1417037,1.333333,1,0,86
...,...,...,...,...,...,...,...,...,...,...
3651,100,40.67,2.45,0.99,-9,19551,4.777667,5,9,117
3652,101,33.85,0.15,0.71,-2,1158,4.833000,5,9,20
3653,102,36.99,0.43,0.79,-3,3393,5.000000,5,9,34
3654,103,30.41,0.24,5.58,-4,1930,5.000000,5,9,18


In [32]:
foreign_keys = [
    {'column': 'champion_ID', 'referenced_table': 'champion', 'referenced_column': 'champion_ID'},
    {'column': 'lane_ID', 'referenced_table': 'lane', 'referenced_column': 'lane_ID'},
    {'column': 'ranking_ID', 'referenced_table': 'ranking', 'referenced_column': 'ranking_ID'}

]



In [33]:
game_stats_create_query = get_create_table_query(
    df_game_stats, ['champion_ID', 'lane_ID', 'ranking_ID'], 'game_stats', foreign_keys
)
game_stats_create_query

'CREATE TABLE game_stats (position INT, win_rate FLOAT, pick_rate FLOAT, ban_rate FLOAT, pbi INT, games_played INT, grade FLOAT, lane_ID INT, ranking_ID INT, champion_ID INT, PRIMARY KEY (champion_ID, lane_ID, ranking_ID), FOREIGN KEY (champion_ID) REFERENCES champion(champion_ID), FOREIGN KEY (lane_ID) REFERENCES lane(lane_ID), FOREIGN KEY (ranking_ID) REFERENCES ranking(ranking_ID))'

In [34]:
execute_query(conn, game_stats_create_query)

Executing


### Matchup

In [35]:
df_matchup = pd.read_excel('datasets_cleaned/matchup.xlsx')
df_matchup

,lane_ID,ranking_ID,type,winrate,games_played,champion_ID,champion_vs_ID
0,1,0,counter,44.01,100862,24,86
1,1,0,counter,44.53,50803,24,420
2,1,0,counter,44.85,28349,24,36
3,1,0,counter,44.86,14508,24,27
4,1,0,counter,45.80,5380,24,154
...,...,...,...,...,...,...,...
30448,5,9,strong,61.82,55,21,43
30449,5,9,strong,60.00,35,21,17
30450,5,9,strong,59.57,47,21,267
30451,5,9,strong,58.06,62,21,117


In [36]:
foreign_keys = [
    {'column': 'champion_ID', 'referenced_table': 'champion', 'referenced_column': 'champion_ID'},
    {'column': 'ranking_ID', 'referenced_table': 'ranking', 'referenced_column': 'ranking_ID'},
    {'column': 'lane_ID', 'referenced_table': 'lane', 'referenced_column': 'lane_ID'},
    {'column': 'champion_vs_ID', 'referenced_table': 'champion', 'referenced_column': 'champion_ID'}
]

In [37]:
matchup_create_query = get_create_table_query(
    df_matchup, ['champion_ID', 'lane_ID', 'ranking_ID', 'champion_vs_ID'], 'matchup', foreign_keys
)

matchup_create_query

'CREATE TABLE matchup (lane_ID INT, ranking_ID INT, type VARCHAR(255), winrate FLOAT, games_played INT, champion_ID INT, champion_vs_ID INT, PRIMARY KEY (champion_ID, lane_ID, ranking_ID, champion_vs_ID), FOREIGN KEY (champion_ID) REFERENCES champion(champion_ID), FOREIGN KEY (ranking_ID) REFERENCES ranking(ranking_ID), FOREIGN KEY (lane_ID) REFERENCES lane(lane_ID), FOREIGN KEY (champion_vs_ID) REFERENCES champion(champion_ID))'

In [38]:
execute_query(conn, matchup_create_query)

Executing


### Insert Values in tables

In [39]:
def get_insert_query(df, table_name):
    # Estrai colonne e valori dal DataFrame
    columns = ', '.join(df.columns)
    values_template = ', '.join(['%s'] * len(df.columns))
    
    # Genera la query di inserimento
    insert_query = f"INSERT INTO {table_name} ({columns}) VALUES ({values_template})"
    escaped_values = []
    # Estrai i valori dal DataFrame come una lista di tuple
    for row in df.values:
        escaped_row = [str(value).replace("'", "''") for value in row]

        escaped_values.append(tuple(escaped_row))
    
    return insert_query, escaped_values

In [40]:
def execute_insert_query(query, values, connection):
    try:
        cursor = connection.cursor()
        cursor.executemany(query, values)
        connection.commit()
        print("Inserimento completato con successo!")
    except Exception as e:
        print(f"Errore durante l'inserimento: {e}")
    finally:
        cursor.close()

### Champion Name

In [41]:
champion_name_insert_query, escaped_values = get_insert_query(df_champion_name, 'champion')
champion_name_insert_query

'INSERT INTO champion (champion_ID, champion_name) VALUES (%s, %s)'

In [42]:
execute_insert_query(champion_name_insert_query, escaped_values, conn)

Inserimento completato con successo!


In [43]:
champion_base_info_insert_query, escaped_values = get_insert_query(df_champion_base_info, 'champion_base_info')
execute_insert_query(champion_base_info_insert_query, escaped_values, conn)

Inserimento completato con successo!


In [44]:
champion_base_stats_insert_query, escaped_values = get_insert_query(df_champion_base_stats, 'champion_base_stats')
execute_insert_query(champion_base_stats_insert_query, escaped_values, conn)

Inserimento completato con successo!


In [45]:
tag_insert_query, escaped_values = get_insert_query(df_tag, 'tag')
execute_insert_query(tag_insert_query, escaped_values, conn)

Inserimento completato con successo!


In [46]:
lane_insert_query, escaped_values = get_insert_query(df_lane, 'lane')
execute_insert_query(lane_insert_query, escaped_values, conn)

Inserimento completato con successo!


In [47]:
ranking_insert_query, escaped_values = get_insert_query(df_ranking, 'ranking')
execute_insert_query(ranking_insert_query, escaped_values, conn)

Inserimento completato con successo!


In [48]:
game_stats_insert_query, escaped_values = get_insert_query(df_game_stats, 'game_stats')
execute_insert_query(game_stats_insert_query, escaped_values, conn)

Inserimento completato con successo!


In [49]:
matchup_insert_query, escaped_values = get_insert_query(df_matchup, 'matchup')
execute_insert_query(matchup_insert_query, escaped_values, conn)

Inserimento completato con successo!
